# Downloading data and unzipping

In [ ]:
# !gdown https://drive.google.com/uc?id=1STTlixkDCgeUMgWqrrYmiwaa3w6B9MFe&export=download

Downloading...
From: https://drive.google.com/uc?id=1STTlixkDCgeUMgWqrrYmiwaa3w6B9MFe
To: /content/assignment_1_data.zip
100% 16.6k/16.6k [00:00<00:00, 14.0MB/s]


In [ ]:
# !unzip assignment_1_data.zip

Archive:  assignment_1_data.zip
   creating: assignment_1_data/
  inflating: assignment_1_data/input.json  
  inflating: assignment_1_data/output.json  


# Importing input data

In [2]:
import json
import re

In [3]:
with open('assignment_1_data/input.json','r', encoding='utf-8') as input_file:
  input_data = json.load(input_file)
  input_file.close()

In [4]:
with open('assignment_1_data/output.json','r', encoding='utf-8') as output_file:
  output_data = json.load(output_file)
  output_file.close()

In [5]:
print(input_data[0])
print(output_data[0])

{'sid': 0, 'input_tokens': ['For', 'every', '100', 'females', 'there', 'were', '90.0', 'males', '.']}
{'sid': 0, 'output_tokens': ['<self>', '<self>', 'one hundred', '<self>', '<self>', '<self>', 'ninety point zero', '<self>', 'sil']}


# Sample functions for generating the output

- If there is a numeric inside the string, the check for the following:
    - If there is no other character except for numerics, then it should be handled like a number.
    - If there is a symbol in front of numerics like symbols of currency, then they should be handled as per currency.
    - If there are any characters, then the string can either be a date or might be having some units associated with the numeric, eg: 5m.

In [6]:
# # Return the text version associated to the numeric in the input string
# def convert_number(input_string):
    

In [64]:
roman_numerals = {
    'I': 'first',
    'II': 'second',
    'III': 'third',
    'IV': 'fourth',
    'V': 'fifth',
    'VI': 'sixth',
    'VII': 'seventh',
    'VIII': 'eighth',
    'IX': 'ninth',
    'X': 'tenth',
    'XI': 'eleventh',
    'XII': 'twelfth',
}

In [1520]:
full_form = {
        'm': 'meter', 
        'mm': 'millimeter',
        'cm': 'centimeter',
        'km': 'kilometer',
        'mi': 'miles',

        'm2': 'square meters',
        'mm2': 'square millimeters',
        'cm2': 'square centimeters',
        'km2': 'square kilometers',
        'mi2': 'square miles',
        
        'sq': 'squares',
        'sqm': 'square meters',
        'sqcm': 'square centimeters',
        'sqkm': 'square kilometers',
        'sqmi': 'square miles',

        'm3': 'cubic meter',
        'mm3': 'cubic millimeter',
        'cm3': 'cubic centimeter',
        'km3': 'cubic kilometer',

        's': 'second',

        'a.m.': 'a m',
        'p.m.': 'p m',

        'M': 'million',
        'm': 'million',
        'mn': 'million',

        'B': 'billion',
        'b': 'billion',
        'bn': 'billion',

        'KB': 'kilobytes',
        'MB': 'megabytes',
        'GB': 'gigabytes',
        'TB': 'terabytes',

        'Kb': 'kilobits',
        'Mb': 'megabits',
        'Gb': 'gigabits',
        'Tb': 'terabits',

        'Kbps': 'kilobits per second',
        'Mbps': 'megabits per second',
        'Gbps': 'gigabits per second',
        'Tbps': 'terabits per second',

        'mA': 'milliamperes'
        }

In [1386]:
numbers = {
    '-': 'minus',
    '0': 'zero',
    '1': 'one',
    '2': 'two',
    '3': 'three',
    '4': 'four',
    '5': 'five',
    '6': 'six',
    '7': 'seven',
    '8': 'eight',
    '9': 'nine',
    '10': 'ten',
    '11': 'eleven',
    '12': 'twelve',
    '13': 'thirteen',
    '14': 'fourteen',
    '15': 'fifteen',
    '16': 'sixteen',
    '17': 'seventeen',
    '18': 'eighteen',
    '19': 'nineteen',
    '20': 'twenty',
    '30': 'thirty',
    '40': 'forty',
    '50': 'fifty',
    '60': 'sixty',
    '70': 'seventy',
    '80': 'eighty',
    '90': 'ninety'
}

In [1387]:
extended_nums = {
    '0': 'ieth',         # y is replaced by this if the last numeric is 0
    '1': 'first',
    '2': 'second',
    '3': 'third',
    '4': 'fourth',
    '5': 'fifth',
    '6': 'sixth',
    '7': 'seventh',
    '8': 'eighth',
    '9': 'ninth'
}

In [1041]:
def is_symbol(input_string):
    if bool(re.search(r'\$', input_string)):
        return 'dollar'
    elif bool(re.search(r'\£', input_string)):
        return 'pound'
    elif bool(re.search(r'\€', input_string)):
        return 'euro'

In [1042]:
is_symbol('€349')

'euro'

- If there are two 

In [1043]:
# Returns if a punctuation is present in the input string
def has_punc(input_string):
    return bool(re.search(r'[^\w\s]', input_string))

In [1044]:
def get_punc(input_string):
    return 'sil'

In [1045]:
# Returns true if the input string has a numeric present
def has_numeric(input_string):
    return bool(re.search(r'\d', input_string))

In [1046]:
# Returns true if the input string has only 4 numerical characters, which would be indicative of an year
def has_fournumeric(input_string):
    return bool(re.search(r'^\d{4}$', input_string))

In [1047]:
get_onlydigits('06')

'six'

In [1240]:
def get_fournumeric(input_string):
    if len(input_string) == 4:
        if input_string[1:3] == '00':               # handles 2007 --> 'two thousand seven'
            return get_onlydigits(input_string)
        else:                                       # handles 1906 --> 'nineteen o six'; but 2012 --> 'twenty twelve'
            first_two = input_string[0:2]
            last_two = input_string[-2:]
            if get_onlydigits(first_two) != 'zero':
                if input_string[2] == '0':              # handles 1906 --> 'nineteen o six';
                    return get_onlydigits(first_two) + ' o ' + get_onlydigits(last_two)
                else:                                   # handles 2012 --> 'twenty twelve';
                    return get_onlydigits(first_two) + ' ' + get_onlydigits(last_two)
            else:                                       # handles 0002 --> 'two';
                return get_onlydigits(last_two)
    else:
        return ''

In [1112]:
get_fournumeric('006')

'NONE'

In [1334]:
# Return true if the input string has only numerical digits
def has_onlydigits(input_string):
    return bool(re.search(r'^(-|\d)\d*[,\d]*\d*$', input_string))

In [1335]:
has_onlydigits('3')

True

In [1336]:
# Return true if the input string has a sequence similar to a ISBN codes that have a dashed sequence.
def has_isbncode(input_string):
    return bool(re.search(r'^\d+[ |-]\d+', input_string))           # Since we want to find a repeating pattern, either the start or the end fixer has to be relaxed

In [1337]:
# The case for dates in the same format still need to be handled
has_isbncode('31-2-2020')

True

In [1338]:
def get_isbncode(input_string):
    res = ''
    for char in input_string:
        # Punctuation case needs to be handled first, else '-' would be taken as 'minus' from the numbers dictionary
        if has_punc(char) or char == ' ':
            res += ' ' + get_punc(char)
            continue
        
        # '0' case needs to be handled before other numerals since this is converted to 'o' rather than 'zero'
        if char == '0':
            res += ' o'
        
        else:
            res += ' ' + numbers[char]
    
    res = ' '.join(res.split())             # to remove extra spaces at the front or end
    return res

In [1339]:
get_isbncode('803.05')

'eight o three sil o five'

In [1563]:
def get_onlydigits(input_string):
    input_string = str(input_string)
    
    # To ensure that no empty strings give any issue
    if input_string.strip() == '':
        return 'NONE'

    res = ''
    input_string = re.sub(r'[^\d-]', '', input_string)
    int_string = int(input_string)
    
    if int_string < 0:
        res += 'minus '                     # there would surely be something in front of the negative sign here.
        input_string = input_string[1:]
        int_string *= -1

    # The case with only 4 digits needs to be incorporated; 1991 --> nineteen ninety one; the first two numerals are separated from the other two

    # Handling numbers directly from dictionary
    if int_string <= 20:
        res += numbers[str(int_string)]     # for example: '07' means '7' and thus has to be looked this way in the dictionary
        return res                          # since we do not need to do anything further from here


    # Handling two digit numbers not in dictionary
    if int_string < 100:
        if str(int_string) in numbers.keys():       # '30' --> 'thirty'
            res += numbers[str(int_string)]
            return res
        else:
            tenth_place = (int_string//10) * 10     # '35' --> 'thirty five'
            unit_place = (int_string % 10)
            res += numbers[str(tenth_place)] + ' ' + numbers[str(unit_place)]
            return res

    t = 1000
    # Handling hundreds
    if int_string < t:
        hundred_place = int_string // 100
        if int_string % 100 == 0:                   # '900' --> 'nine hundred'
            res += numbers[str(hundred_place)] + ' hundred'
            return res
        else:
            res += numbers[str(hundred_place)] + ' hundred ' + get_onlydigits(str(int_string % 100))
            return res

    # Handling thousands
    # As the number increases, the number of zeroes becomes over-whelming! Better to convert it into a variable
    m = t*t
    if int_string < m:
        thousand_place = int_string // t
        if int_string % t == 0:                   
            res += get_onlydigits(str(thousand_place)) + ' thousand' # '72000' --> 'seventy two thousand'; and 72 is not in the main dictionary and thus recursion needs to be done
            return res
        else:
            res += get_onlydigits(str(thousand_place)) + ' thousand ' + get_onlydigits(str(int_string % t))
            return res

    # Handling millions
    b = m*t
    if int_string < b:
        million_place = int_string // m
        if int_string % m == 0:                   
            res += get_onlydigits(str(million_place)) + ' million' # '72000' --> 'seventy two thousand'; and 72 is not in the main dictionary and thus recursion needs to be done
            return res
        else:
            res += get_onlydigits(str(million_place)) + ' million ' + get_onlydigits(str(int_string % m))
            return res

    # Handling billions
    tr = b*t
    if int_string < tr:
        billion_place = int_string // b
        if int_string % b == 0:                   
            res += get_onlydigits(str(billion_place)) + ' billion' # '72000' --> 'seventy two thousand'; and 72 is not in the main dictionary and thus recursion needs to be done
            return res
        else:
            res += get_onlydigits(str(billion_place)) + ' billion ' + get_onlydigits(str(int_string % b))
            return res

    # Handling trillions
    pr = tr*t
    if int_string < pr:
        prillion_place = int_string // tr
        if int_string % tr == 0:                   
            res += get_onlydigits(str(prillion_place)) + ' trillion' # '72000' --> 'seventy two thousand'; and 72 is not in the main dictionary and thus recursion needs to be done
            return res
        else:
            res += get_onlydigits(str(prillion_place)) + ' trillion ' + get_onlydigits(str(int_string % tr))
            return res

    # Check if numbers greater than billions appear in the corpus often; ref: GDP is also only in trillions!

    # Symbols need to be incorporated; $20 --> '20 dollars'; $1 --> 'one dollar'


    # Check if the numbers can also come with thousand separators like 1,000,000

    # 'res +=' can also be removed since we are not making any use of that operation!

    return res

In [1564]:
' '.strip()

''

In [1546]:
get_onlydigits(21)

'twenty one'

In [1547]:
get_onlydigits(19)

'nineteen'

In [1343]:
def has_decimal(input_string):
    return bool(re.search(r'^\d+[.]\d+$', input_string))      # ending on a digit is justified since period (.) is a different token in the corpus

In [1344]:
has_decimal('0.13.')

False

In [1345]:
a = '0.153'
a.split('.')

['0', '153']

In [1346]:
def get_decimal(input_string):
    res = ''
    num_list = input_string.split('.')
    res = res + get_onlydigits(num_list[0]) + ' point ' + get_isbncode(num_list[1])
    return res

In [1347]:
get_decimal('31212.14')

'thirty one thousand two hundred twelve point one four'

In [1348]:
def has_units(input_string):
    return bool(re.search(r'^\d+.*[a-zA-Z]+', input_string))

In [1409]:
get_units('100 KB/s')

'per second'

In [1414]:
re.split(r'\s+|[ /]\s*', '100 KB/s')

['100', 'KB', 's']

In [1416]:
'100Kb/s'.split(' ')

['100Kb/s']

In [1423]:
re.findall("([0-9]+|[A-Z]+)", "5GB")

['5', 'GB']

In [1576]:
def get_units(input_string):
    res = ''
    is_per = False
    is_space = False
    
    # '50.0Kb Gb/s' --> ['50.0Kb', 'Gb/s']
    words_list = input_string.split(' ')

    # if bool(re.search(r'/', input_string)):
    #     is_per = True

    # if re.search(r'/[a-zA-Z]', input_string):       # for ex: '70.05/sqkm' --> 'seventy point zero five per square kilometer'
    #     words_list = re.split(r'\s+|[ /]\s*', input_string)
    #     is_per = True
    
    # if not is_per:
    #     words_list = input_string.split(' ')
    #     is_space = True

    for word in words_list:
        # Separating the number; for ex: '50.0Kb' --> ['50.0', 'Kb']
        key_list = re.findall('([0-9.,/]+|[a-zA-Z].+)', word)
        for key in key_list:
            # print(key)
            if bool(re.search(r'/', key)):
                temp_list = key.split('/')
                left = True
                for temp in temp_list:
                    # print(temp)
                    if not left:
                        res += 'per '
                    # if temp == '':
                    #     continue
                    if bool(re.search(r'\.', temp)):
                        print(temp)
                        res += get_decimal(temp) + ' '
                    elif temp in full_form.keys():
                        res += full_form[temp] + ' '
                    else:
                        res += temp + ' '
                    left = False
            elif bool(re.search(r'\.', key)):
                print(key)
                res += get_decimal(key) + ' '
            elif bool(re.search(r'^\d+$', key)):
                res += get_onlydigits(key) + ' '
            elif key in full_form.keys():
                # print(key)
                res += full_form[key] + ' '
            else:
                res += key + ' '

    return ' '.join(res.split())

    #     temp_list = word.split('/')                 # for ex: 100 Kb/s
    #     if len(temp_list) == 1:

    #     # If '/' divides a number by a number
    #     left = True                                 # if True, then the left part of '/' is being processed
    #     for temp in temp_list:
    #         # Handling the abbreviations of units
    #         if word in full_form.keys():
    #             if is_per:                                      # this would tell us if there was a '/' symbol followed by a letter or not
    #                 res += ' per ' + full_form[word] + ' ' 
    #             else:
    #                 res += full_form[word] + ' '

    #         # Handling the numerical part
    #         elif has_decimal(word):
    #             res += get_decimal(word) + ' '
            
    #         elif has_onlydigits(word):
    #             res += get_onlydigits(word) + ' '
    
    # res = ' '.join(res.split())             # to remove extra spaces at the front or end
    # return res

In [1577]:
re.findall('([0-9.,/]+|[a-zA-Z].+)', '100')

['100']

In [1519]:
get_units('100.4/Gbps')

100.4/
100.4

Gbps


'one hundred point four per gigabits per second'

In [1517]:
get_units('179.4/km2')

179.4/
179.4

km2


'one hundred seventy nine point four per square kilometers'

In [1501]:
get_units('823.08 KB')

823.08
KB


'eight hundred twenty three point o eight kilobytes'

In [1355]:
def has_extendednums(input_string):
    return bool(re.search(r'\d+[ ]*(st|nd|rd|th)', input_string))

In [1356]:
'00209'.lstrip('0')

'209'

In [1357]:
get_extendednums('9')

'ninth'

In [1521]:
# Handles the cases of extended numerals; for ex: '100th' --> 'one hundredth'
def get_extendednums(input_string):
    # Getting the last numeric digit present in the string
    numeric = re.sub(r'[^\d]', '', input_string)
    non_unit_numeric = (int(numeric) // 10) * 10

    input_string = input_string.lstrip('0')         # to remove initial zeroes in the string
    # For ex: 02nd --> 2, then this only has one digit
    if input_string in extended_nums.keys():
        return extended_nums[input_string]
    
    unit_numeric = numeric[-1]

    # If the last two digits are between 10 and 20 (both inclusive), then we have to get its direct form
    last_two_digits = int(numeric[-2:])
    if len(numeric) == 2 and (last_two_digits >= 10) and (int(last_two_digits) < 20):
        return get_onlydigits(str(last_two_digits)) + 'th'
    elif len(numeric) > 2 and (last_two_digits >= 10) and (int(last_two_digits) < 20):
        return get_onlydigits((int(numeric) // 100) * 100) + ' ' + get_extendednums(str(last_two_digits))
    else:
        # Getting the translation for the non-last digit
        non_unit_string = get_onlydigits(str(non_unit_numeric))
        # Handling case with last two digits as 0; for ex: 100th --> 'one hundreth' and not 'one hundreith'
        if last_two_digits == 0:
            return get_onlydigits((int(numeric) // 100) * 100) + 'th'
        # Handling case with last two digits as 0; for ex: 130th --> 'one hundred thirieth' and not 'one hundred thirtyth'
        elif unit_numeric == '0':
            return non_unit_string[:-1] + extended_nums[unit_numeric]
        else:
            return non_unit_string + ' ' + extended_nums[unit_numeric]

In [1522]:
get_extendednums('20')

'twentieth'

In [1523]:
def has_allcaps(input_string):
    return bool(re.search(r'^[A-Z]+$', input_string))

In [1524]:
# Converst the abbreviated version to a spaced version as per output format
def get_allcaps(input_string):                          # ref: sid 202
    return ((' '.join(input_string)).lower())

In [1525]:
# Returns true if the input string forms a part of an abbreviation
def get_abbr(input_string):
    if input_string in full_form.keys():
        return full_form[input_string]

In [1526]:
# bool(re.search(r'[A-Z]\.', input_string)

In [1527]:
get_abbr('sq')

'squares'

In [1528]:
has_numeric('He has #22# dogs')

True

In [1529]:
def has_date(input_string):
    month_day = bool(re.search(r'Jan|January|Feb|February|Mar|March|Apr|April|May|Jun|June|Jul|July|Aug|August|Sept|September|Oct|October|Nov|November|Dec|December', input_string))
    ddmmyyyy = bool(re.search(r'^\d{1,2}[ ,-]+([0]\d{1}|1[012])[ ,-]\d{4}$', input_string))     # no control on daters
    mmddyyyy = bool(re.search(r'^([0]\d{1}|1[012])[ ,-]\d{1,2}[ ,-]+\d{4}$', input_string))
    yyyymmdd = bool(re.search(r'^\d{4}[ ,-]+([0]\d{1}|1[012])[ ,-]\d{1,2}$', input_string))
    
    # Starting with year
    if yyyymmdd:
        year = True
    else:
        year = False

    # Starting with month
    if mmddyyyy:
        month = True
    else:
        month = False
    
    # Starting with day
    if ddmmyyyy:
        day = True
    else:
        day = False
    
    # Has a day component
    if yyyymmdd or mmddyyyy or ddmmyyyy or month_day:
        return True, [day, month, year, month_day]
    else:
        return False, [False, False, False, False]

In [1530]:
has_date('20 May.')

(True, [False, False, False, True])

In [1531]:
bool(re.search(r'^\D+$','January eiufha askefh auet;sadfl iasf/2'))

False

In [1532]:
bool(re.search(r'^\d{2}', '20 Jan'))

True

In [1533]:
re.split(r'\s+|[,.-]\s*', '20 May.2')

['20', 'May', '2']

In [1534]:
month = {
    'jan': 'january',
    'feb': 'february',
    'mar': 'march',
    'apr': 'april',
    'may': 'may',
    'jun': 'june',
    'jul': 'july',
    'aug': 'august',
    'sept': 'september',
    'oct': 'october',
    'nov': 'november',
    'dec': 'december',
}

In [1535]:
month_num = {
    '01': 'january',
    '02': 'february',
    '03': 'march',
    '04': 'april',
    '05': 'may',
    '06': 'june',
    '07': 'july',
    '08': 'august',
    '09': 'september',
    '10': 'october',
    '11': 'november',
    '12': 'december',
}

In [1536]:
' '.join(' 20 Jan '.split())

'20 Jan'

In [1555]:
def get_date(input_string, order):
    # order is of the form [day, month, year, month_day]

    def has_month(input_string):
        return bool(re.search(r'^\D+$', input_string))

    def get_month(input_string):
        if input_string.lower() in month.keys():
            return month[input_string.lower()]
        else:
            return input_string.lower()

    res = ''
    keyword_list = re.split(r'\s+|[,.-]\s*', input_string)

    # Back-filling day, month or year number with 0s; for ex: October 1, 2021 --> October 01, 2021
    for i in range(len(keyword_list)):
        if len(keyword_list[i]) == 1:
            keyword_list[i] = '0' + keyword_list[i]
        # elif len(keyword_list[i]) == 3:
        #     keyword_list[i] = '0' + keyword_list[i]

    # for keyword in keyword_list:
    #     print(keyword)

    # January 14, 2008 --> january fourteenth two thousand eight
    # 20 January 2010 --> the twentieth of january twenty ten
    # January 2010 --> january twenty ten

    if order[3]:
        # Handles if input string only has the month name like 'January'
        if bool(re.search(r'^\D+$',input_string)):
            return '<self>'                 # since it has no numeral to tell us that we are talking about a specific date
        else:
            # If first keyword is a month name
            if bool(re.search(r'^\D+$', keyword_list[0])):
                if has_month(keyword_list[0]):
                    res += get_month(keyword_list[0])
                
                # If the next keyword is a date, then push its extended form                  
                if bool(re.search(r'^\d{2}$', keyword_list[1])):
                    res += ' ' + get_extendednums(keyword_list[1])
                    date_inlcuded = True

                    # If there exists an year to process
                    if len(keyword_list) > 2:
                        res += ' ' + get_fournumeric(keyword_list[2])
                    
                    return ' '.join(res.split())

                # If the next keyword is a year
                elif bool(re.search(r'^\d{4}$', keyword_list[1])):
                    res += ' ' + get_fournumeric(keyword_list[1])
                    return ' '.join(res.split())


            # If the keyword is a date, then its extended form needs to be used; 20 --> the twentieth
            elif bool(re.search(r'^\d{2}$', keyword_list[0])):
                res = 'the ' + get_extendednums(keyword_list[0]) + ' of '
                date_included = True

                # if has_month(keyword_list[1]):
                res += get_month(keyword_list[1])
                month_included = True

                # If there exists an year to process
                if len(keyword_list) > 2:
                    res += ' ' + get_fournumeric(keyword_list[2])
                    year_included = True
                
                return ' '.join(res.split())
    
    # # If the date is presented in ddmmyyyy format
    # elif order[0]:
    #     res = getkeyword_list[0]


In [1556]:
get_date('2 July 2020', [False, False, False, True])

'the second of july twenty twenty'

In [1565]:
#todo: write your own solution.
def solution(input_tokens):
  sol = []
  flag = False

  for token in input_tokens:

    # All the extra spaces at the start and end of the token should be trimmed since the tokenisation may have a problem (for ex: sid 85 --> "21 ")
    token = ' '.join(token.split())
    
    # Handling Punctuation.
    if((has_punc(token) and (len(token) == 1)) and not flag):
      sol.append(get_punc(token))
      flag = True
    
    # Handling roman numerals.
    if token in roman_numerals.keys() and not flag:
      sol.append('the ' + roman_numerals[token])
      flag = True

    # Handling all capital tokens (not as abbreviations though).
    if has_allcaps(token) and (len(token) > 1) and not flag:
      sol.append(get_allcaps(token))
      flag = True
    
    # Handling dates.
    if has_date(token)[0] and not flag:
      res = get_date(token, has_date(token)[1])
      if res != 'NONE':
        sol.append(res)
        flag = True

    # Handling 4 digits of numerals; years in general, if there are no symbols, before other numerical strings.
    if has_fournumeric(token) and not flag:
      sol.append(get_fournumeric(token))
      flag = True

    # Handling pure numericals (numbers).
    if has_onlydigits(token) and not flag:
      sol.append(get_onlydigits(token))
      flag = True
    
    ## Date has to be handled here

    # Handling extended numericals, for ex: 21st --> 'twenty first'; has to be handled before units.
    if has_extendednums(token) and not flag:
      sol.append(get_extendednums(token))
      flag = True

    # Handling tokens having numeric character with units (has to be done before ISBN; for ex: 823.05 KB).    
    if has_units(token) and not flag:
      sol.append(get_units(token))
      flag = True

    # Handling ISBN Code.
    if has_isbncode(token) and not flag:
      sol.append(get_isbncode(token))
      flag = True

    # Handling decimal values.
    if has_decimal(token) and not flag:
      sol.append(get_decimal(token))
      flag = True

    # If nothing else, then the token should be <self>.
    if not flag:
      sol.append('<self>')

    flag = False    # preparing for the next iteration.

  return sol 


In [1566]:
solution(['21112-2302-2121', '3.1', '823.06 KB', '50,000','21st', '1996', '12 July 2008','3'])

823.06
KB


['two one one one two sil two three o two sil two one two one',
 'three point one',
 'eight hundred twenty three point o six kilobytes',
 'fifty thousand',
 'twenty first',
 'nineteen ninety six',
 'the twelveth of july two thousand eight',
 'three']

In [1567]:
has_units('823.06 KB')
get_units('823.06 KB')

823.06
KB


'eight hundred twenty three point o six kilobytes'

In [1568]:
' '.join('21 '.split())

'21'

In [1578]:
def solution_dump(solution_file_path):
  solution_data = []
  for input_sentence in input_data:
    solution_sid = input_sentence['sid']
    print(solution_sid)
    solution_tokens = solution(input_sentence['input_tokens'])
    solution_data.append({'sid':solution_sid,
                          'output_tokens':solution_tokens})

  with open(solution_file_path,'w') as solution_file:
    json.dump(solution_data, solution_file, indent=2, ensure_ascii=False)
    solution_file.close()

In [1579]:
solution_dump('solution.json')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
p.m.


ValueError: invalid literal for int() with base 10: ''

## Error in Solution wrt Ground Truth Data

In [1395]:
with open('solution.json','r', encoding='utf-8') as solution_file:
  solution_data = json.load(solution_file)
  solution_file.close()

In [1396]:
# Finding count of differences
error_list = {}
count = 0
percentage_error = 0
total_perc_error = 0

for output_sentence in output_data:
    res_sid = output_sentence['sid']
    res_token = output_sentence['output_tokens']

    sol_token  = solution_data[res_sid]['output_tokens']
    for i in range(len(res_token)):
        if (res_token[i] != sol_token[i]):
            count += 1
    percentage_error = 100*count/len(res_token)
    total_perc_error += percentage_error
    error_list[res_sid] = percentage_error
    count = 0

error_list['total'] = total_perc_error/len(error_list)

In [1397]:
with open('error_list.json','w') as error_file:
    json.dump(error_list, error_file, indent=2, ensure_ascii=False)
    error_file.close()

# Expected submission

In [1181]:
'''
We expect a .py file which takes as input the input file path and the solution file path:

python run_assignment1.py --input_path <path_to_input> --solution_path <path_to_solution>

Once the solution file is generated, we will run a checker script to get the performance metrics:

python run_checker.py --solution_path <path_to_solution> --ground_truth_path <path_to_ground_truth>

'''

'\nWe expect a .py file which takes as input the input file path and the solution file path:\n\npython run_assignment1.py --input_path <path_to_input> --solution_path <path_to_solution>\n\nOnce the solution file is generated, we will run a checker script to get the performance metrics:\n\npython run_checker.py --solution_path <path_to_solution> --ground_truth_path <path_to_ground_truth>\n\n'

# Running on HPC

In [ ]:
'''
Students may use the hpc run their code. Please use the following commands to create a conda environment:


module load apps/anaconda/3EnvCreation
conda create --prefix=~/<environment_name> python=3.6
module unload apps/anaconda/3EnvCreation

This is a one-time process. Once the environment is created, the students may wish to install some packages. This can be done as follows:

module load apps/anaconda/3
conda activate ~/<environment_name>
module unload apps/anaconda/3
pip install <package_name> 

It is highly recommended that the students do not install PyTorch in this fashion. The recommended way of configuring PyTorch will be discused in the PyTorch tutorial class scheduled for later this semester.

This assignment will be entirely run on CPU. An interactive CPU job can be run as follows:

qsub -I -P <project_name> -l select=1:ncpus=1:mem=16G -l walltime=<hh:mm:ss>

Once the job is ready, the conda environment would have to be activated again as follows:

module load apps/anaconda/3
conda activate ~/<environment_name>
module unload apps/anaconda/3

Students may be interested to run a jupyter notebook on HPC. Kindly follow the steps from the link below:

https://supercomputing.iitd.ac.in/?softevents=jupyternotebook

'''